In [3]:
from datasets import load_dataset
from transformers import (
    ViltProcessor,
    DefaultDataCollator,
    ViltForQuestionAnswering,
    TrainingArguments,
    Trainer,
)
import torch
from PIL import Image

dataset = load_dataset("json", data_files="data/train.jsonl", split="train")


answers = [item for item in dataset["answer"]]
unique_labels = list(set(answers))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}


model = "dandelin/vilt-b32-finetuned-vqa"
processor = ViltProcessor.from_pretrained(model)


In [4]:
def preprocess_data(examples):
    pids = examples["pid"]

    image_paths = [
        f"data/train_fill_in_blank/train_fill_in_blank/{pid}/image.png" for pid in pids
    ]

    images = [Image.open(image_path) for image_path in image_paths]
    texts = examples["question"]
    # try:
    #     for i in range(0, len(texts)):
    #         encoding = processor(
    #             images[i], texts[i], padding="max_length", truncation=True, return_tensors="pt"
    #         )
    # except:
    #     print(image_paths[i])
    #     print(texts[i])
        
    try:
        encoding = processor(
            images, texts, padding="max_length", truncation=True, return_tensors="pt"
        )
    except Exception as e:
        print(f"Error {e} in processor , will skip this batch")
        return {
            "input_ids": [],
            "token_type_ids": [],
            "attention_mask": [],
            "pixel_values": [],
            "pixel_mask": [],
        }

    for k, v in encoding.items():
        encoding[k] = v.squeeze()

    targets = []

    for answer in examples["answer"]:
        target = torch.zeros(len(id2label))
        answer_id = label2id[answer]
        target[answer_id] = 1.0
        targets.append(target)
    encoding["labels"] = targets
    # print(encoding.keys())
    
    return encoding


In [9]:
processed_dataset = dataset.map(
    preprocess_data,
    batched=True,
    remove_columns=[
        "question",
        "answer",
        "ques_type",
        "grade",
        "label",
        "pid",
        "hint",
        "unit",
    ],
)


Map:   0%|          | 0/14549 [00:00<?, ? examples/s]

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', '

In [ ]:
processed_dataset

In [ ]:

data_collator = DefaultDataCollator()
model = ViltForQuestionAnswering.from_pretrained(
    model,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

In [5]:
training_args = TrainingArguments(
    output_dir="model/baseline",
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_steps=200,
    logging_steps=200,
    learning_rate=5e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="tensorboard",
)

split_ratio = 0.9
split_idx = round(len(processed_dataset) * split_ratio)
train_ds = processed_dataset.select(range(split_idx))
eval_ds = processed_dataset.select(range(split_idx, len(processed_dataset)))


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=processor,
)



In [ ]:
# trainer.train()

/home/congnguyen/miniconda3/envs/myenv/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [11]:
processor = ViltProcessor.from_pretrained("model/baseline20epoch/checkpoint-4000")
model = ViltForQuestionAnswering.from_pretrained("model/baseline20epoch/checkpoint-4000")

In [16]:
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import json

# prepare image + question
eval_dir = os.path.abspath("data/test_data/test_data/")
eval_data = {}
for it in listdir(eval_dir):
    eval_data.update({it: []})
    for f in listdir(join(eval_dir, it)):
        file = join(eval_dir, it, f)
        eval_data[it].append(file)

def get_data(idx):
    img_url = eval_data[idx][0]
    image = Image.open(img_url)
    text_url = eval_data[idx][1]
    textf = open(text_url)
    # returns JSON object as 
    # a dictionary
    data = json.load(textf)
    text = data["question"]
    return image, text

# # prepare inputs
result = {}
for id in eval_data:
    image, text = get_data(id)
    encoding = processor(image, text, return_tensors="pt")
    # forward pass
    outputs = model(**encoding)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    result.update({id: model.config.id2label[idx]})
    # print(idx, " - ",text, " Answer: ",  model.config.id2label[idx], logits[0][idx])


315  -  Move the ruler to measure the length of the line to the nearest centimeter. The line is about (_) centimeters long.  Answer:  5 tensor(4.9244, grad_fn=<SelectBackward0>)
388  -  Use dice to measure the line. The line is about (_) dice long.  Answer:  4 tensor(4.9159, grad_fn=<SelectBackward0>)
367  -  How many cinnamon rolls are there?  Answer:  20 tensor(3.1017, grad_fn=<SelectBackward0>)
23  -  How many cubes long is the tape?  Answer:  3 tensor(5.4721, grad_fn=<SelectBackward0>)
389  -  There are 7 balloons in the top ten frame. How many balloons are in the bottom ten frame?  Answer:  6 tensor(4.4958, grad_fn=<SelectBackward0>)
389  -  Move the ruler to measure the length of the line to the nearest centimeter. The line is about (_) centimeters long.  Answer:  6 tensor(4.6259, grad_fn=<SelectBackward0>)
485  -  How many shapes are on the frame?  Answer:  8 tensor(3.4253, grad_fn=<SelectBackward0>)
23  -  How many shapes are green?  Answer:  3 tensor(5.2342, grad_fn=<SelectBac

In [205]:
len(eval_data)

3638

In [202]:
import csv

with open('result.csv', 'w', newline='') as csvfile:
    fieldnames = ['ID', 'Label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for idx in result:
        writer.writerow({'ID': idx, 'Label': result[idx]})


In [169]:
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
import json
import os 
from os import listdir
from os.path import isfile, join
from PIL import Image
import torchvision.transforms as transforms

class VQA_Data(Dataset):
    def __init__(self, datadir, transform=None, target_transform=None):
        self.datadir = os.path.abspath(datadir)
        self.images, self.annotations = self.__get_file_from_path__()
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.target_transform = target_transform

    def __get_file_from_path__(self):
        """
            data: {id: _img_, _annotations_}
            images = [],
            annatations = []
        """
        # data = {}
        # images, annotations = {}, {}
        images, annotations = [], []
        for it in listdir(self.datadir):
            if isfile(join(self.datadir, it)):
                continue
            else:
                for f in listdir(join(self.datadir, it)):
                    file = join(self.datadir, it, f)
                    if "image" in f:
                        # images.update({self.__get_id__(file): file})
                        images.append(file)
                    else:
                        # annotations.update({self.__get_id__(file): file})
                        annotations.append(file)
        return images, annotations
                        
    def __len__(self):
        return len(self.images)
        
    def __get_id__(self, file_path):
        id = file_path.split("/")[-2]
        return id
        
    def __getitem__(self, idx):
        img_path = self.images[idx]
        annotation_path = self.annotations[idx]
        image = self.read_image(img_path)
        print(image.shape)
        # label = self.img_labels.iloc[idx, 1]
        # if self.transform:
        #     image = self.transform(image)
        #     print(image.shape)
        # if self.target_transform:
        #     annotation_path = self.target_transform(annotation_path)
        return image
        
    def read_image(self, image_path):
        return Image.open(image_path)
        

In [170]:
training_data = VQA_Data("data/train_fill_in_blank/train_fill_in_blank")

In [171]:
train_dataloader = DataLoader(training_data, batch_size=4, shuffle=True)

In [172]:
next(iter(train_dataloader))

AttributeError: shape

In [11]:
Image.open("data/train_fill_in_blank/train_fill_in_blank/102332/image.png")